In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/home/takosaga/hadoop-2.9.2"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["HADOOP_HOME"], "bin")

In [2]:
%%writefile temperature_mapper.py
#!/usr/bin/env python3

# for interacting with the interpreter
import sys

# for input stream to python read line by line
for line in sys.stdin:
    # line stips ends
    line = line.strip()
    # value gets into two variables of city and temp
    city,temp = line.split(',')

    try:
        temp = float(temp) # try to convert temp to a float
        print('%s\t%s' % (city, temp))
    except ValueError:
        continue # if temp cant be converted then continue


Writing temperature_mapper.py


In [3]:
%%writefile temperature_reducer.py
#!/usr/bin/env python3

import sys

# for counters for citys
current_city = None
current_sum = 0
current_count = 0

for line in sys.stdin:
    line = line.strip()
    city, temp = line.split('\t',1) # since a tab was used in mappers, uses it as delimiter
    try:
        temp = float(temp)
    except ValueError:
        continue # double sanity check to see temp is a float

    if current_city == city:
        #counting
        current_sum += temp
        current_count += 1
    else:
        if current_city:
            # Output the avg temp for the current city
            print('%s\t%s'%(current_city,current_sum/current_count))
        #if not current city then current city changes
        current_city = city
        current_sum = temp
        current_count = 1

# output the average temp for the last city
if current_city == city:
    print('%s\t%s' % (current_city, current_sum / current_count))

Writing temperature_reducer.py


In [4]:
!chmod +x mapper.py reducer.py

In [5]:
%%writefile temperature_data.txt
NewYork,23.5
NewYork,25.1
NewYork,22.3
LosAngeles,30.0
LosAngeles,29.8
LosAngeles,31.2
Chicago,15.5
Chicago,14.8
Chicago,16.2

Writing temperature_data.txt


In [7]:
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar \
-files temperature_mapper.py,temperature_reducer.py -mapper temperature_mapper.py -reducer temperature_reducer.py -input temperature_data.txt -output output

24/10/19 16:48:39 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
24/10/19 16:48:39 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
24/10/19 16:48:39 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
24/10/19 16:48:39 INFO mapred.FileInputFormat: Total input files to process : 1
24/10/19 16:48:39 INFO mapreduce.JobSubmitter: number of splits:1
24/10/19 16:48:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local944594191_0001
24/10/19 16:48:39 INFO mapred.LocalDistributedCacheManager: Localized file:/home/takosaga/Projects/fall_24/big_data/hadoop/temperature_mapper.py as file:/tmp/hadoop-takosaga/mapred/local/1729345719618/temperature_mapper.py
24/10/19 16:48:39 INFO mapred.LocalDistributedCacheManager: Localized file:/home/takosaga/Projects/fall_24/big_data/hadoop/temperature_reducer.py as file:/tmp/hadoop-takosaga/mapred/loca

In [9]:
!cat output/part-00000

Chicago	15.5
LosAngeles	30.333333333333332
NewYork	23.633333333333336
